In [1]:
import pandas as pd
import numpy as np

import networkx as nx
from networkx.algorithms.community import k_clique_communities
import random as rd
rd.seed(42)

import warnings
warnings.filterwarnings('ignore')

## Random Walk Sampling Method 2
### Sample 9

In [2]:
G_rwm2_9 = nx.read_gml(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\Samples\Random Walk Sampling\G_rw2_9.gml')

In [3]:
# Confirm that Graph is Directed
nx.is_directed(G_rwm2_9)

True

In [4]:
# Display Size of the Graph
print(nx.info(G_rwm2_9))

MultiDiGraph with 50000 nodes and 132739 edges


Confirmed that the graph meets the criterea of having more edges than nodes and maintained the directed structure.

In [5]:
# List all nodes in the sample
n_rwm2_9 = list(G_rwm2_9.nodes)

In [6]:
# Get node properties from data exploration exercise for comparison
node_properties = pd.read_csv(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\node_properties_cl.csv')
node_properties[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

,degree,in_degree,out_degree,eigen_centrality
count,4.666873e+06,4.666873e+06,4.666873e+06,4.666873e+06
mean,4.494298e+00,2.247149e+00,2.247149e+00,3.970074e-05
std,3.935922e+01,1.998412e+01,2.868249e+01,4.611943e-04
min,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
25%,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
50%,2.000000e+00,1.000000e+00,1.000000e+00,4.892620e-20
75%,3.000000e+00,1.000000e+00,2.000000e+00,1.517475e-14
max,2.791700e+04,1.211600e+04,2.533500e+04,1.642653e-01


In [7]:
rwm2_9_df = nx.to_pandas_edgelist(G_rwm2_9) 
rwm2_9_df.head()

,source,target,weight
0,14Te5Ynvh4PhExXBxgog4A2527MdMLnRJC,1Fiiw52wqmeJZBPaYS1Sk6ReHbRKGcLgbM,1.0
1,3AfNAPBoetKLenPvBKn7cyHCcKZcWEAXdz,3Jdi85yNP6eYFkSeMgzERN9QZDY4W7s36J,1.0
2,1ByRdneESvXjE4r74BKm9owf1EtpaHGcER,123CuAkddvb8A7PC54GcwfcTPcM3EzKfnK,1.0
3,1NS35ayQpn39sYhiET3DsNXBL77MKs9yX,18XfgTS9DWrL3z465TvYbhcYXUs9L3GT5v,1.0
4,1NS35ayQpn39sYhiET3DsNXBL77MKs9yX,1CB2T1zwJfD1qbjNmiMXLvgiCKEykhSSwa,1.0


In [8]:
# Number of unique input addresses
rwm2_9_df['source'].nunique()

32160

In [9]:
# Number of unique output addresses
rwm2_9_df['target'].nunique()

31280

We observe relative balance between input and output addresses, with slightly more input addresses. In the full dataset there is also relative balance, but there were more output addresses. 

In [10]:
rwm2_9_df.describe()

,weight
count,132739.000000
mean,0.778129
std,0.494387
min,-69.648093
25%,0.526627
50%,1.000000
75%,1.000000
max,52.821905


The weight variable shows that the distribution of the weighting (amount*0.5 + timestamp*0.5) is similar to that of overall transaction amounts.

In [11]:
# Subset by fraudulent/high-risk nodes
fr_nodes = node_properties[node_properties['fraud_flag'] == 1]
fr_nodes = fr_nodes['address'].tolist()
print('Total number of fraudulent/high-risk nodes : ', len(fr_nodes))

# Check for number of fraudulent/high-risk addresses in the sample
print('Number of fraudulent/high-risk nodes in sample : ', len(set(n_rwm2_9).intersection(fr_nodes)))

Total number of fraudulent/high-risk nodes :  4563
Number of fraudulent/high-risk nodes in sample :  308


These numbers suggest that random selection of nodes worsens the class imbalance problem, but when considering the counts relative to the size of the network the issue is less pronounced.

Full Network: 0.11% 

Sample: 0.62%

## Node Properties

In [12]:
# Get Properties

address = [node for (node, val) in G_rwm2_9.degree()]
degree = [val for (node, val) in G_rwm2_9.degree()]
in_degree = [val for (node, val) in G_rwm2_9.in_degree()]
out_degree = [val for (node, val) in G_rwm2_9.out_degree()]
#eigen_centrality = nx.eigenvector_centrality(G_rwm2_9) # NetworkXNotImplemented: not implemented for multigraph type
#eigen_centrality = [eigen_centrality[node] for node in eigen_centrality]

In [13]:
# Add Properties to DataFrame

nodes = pd.DataFrame()
nodes['address'] = address
nodes['degree'] = degree
nodes['in_degree'] = in_degree
nodes['out_degree'] = out_degree
#nodes['eigen_centrality'] = eigen_centrality

In [14]:
# Add fraud flag

nodes = nodes.assign(**dict.fromkeys(['fraud_flag'], 0))

for i in fr_nodes:
    nodes.loc[nodes.address == i, 'fraud_flag'] = 1
    
nodes.head()

,address,degree,in_degree,out_degree,fraud_flag
0,14Te5Ynvh4PhExXBxgog4A2527MdMLnRJC,1,0,1,0
1,3AfNAPBoetKLenPvBKn7cyHCcKZcWEAXdz,1,0,1,0
2,1ByRdneESvXjE4r74BKm9owf1EtpaHGcER,2,1,1,0
3,1M27uUBFwB9p9wU3qGTAGGkoKPRGZzUpmL,1,1,0,0
4,1NS35ayQpn39sYhiET3DsNXBL77MKs9yX,8,3,5,0


In [15]:
# Describe sample properties overall

nodes[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,50000.000000,50000.000000,50000.000000
mean,5.309560,2.654780,2.654780
std,38.312194,16.754514,25.145962
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000
75%,4.000000,2.000000,2.000000
max,4092.000000,1399.000000,2693.000000


In [16]:
# Describe Properties of Fraudulent Nodes

nodes_fraud = nodes[nodes['fraud_flag'] == 1]
nodes_fraud[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,308.000000,308.000000,308.000000
mean,122.512987,50.337662,72.175325
std,438.190501,170.940099,293.877351
min,1.000000,0.000000,0.000000
25%,6.000000,2.000000,1.000000
50%,21.000000,9.000000,7.000000
75%,77.500000,35.250000,32.000000
max,4092.000000,1399.000000,2693.000000


In [17]:
# Describe Properties of Non-Fraudulent Nodes

nodes_licit = nodes[nodes['fraud_flag'] == 0]
nodes_licit[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,49692.000000,49692.000000,49692.000000
mean,4.583112,2.359233,2.223879
std,14.316856,9.367169,8.516112
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000
75%,4.000000,2.000000,2.000000
max,924.000000,923.000000,534.000000


For all 3 cases the degree distribution properties are similar to the overall network.

## Graph Properties

In [18]:
# Density
nx.density(G_rwm2_9) # Connectivity remains very low

5.3096661933238664e-05

In [19]:
# Communities - k-Cliques

com_generator_rwm2_9 = k_clique_communities(G_rwm2_9.to_undirected(), k=10)
com_rwm2_9 = next(com_generator_rwm2_9)
print('The number of communities in the sample : ' + str(len(com_rwm2_9)))

The number of communities in the sample : 10


The sample has a variety of community structures. (Note: Giant Component = 107)